In [1]:
from ultralytics import YOLO
from utils import *
from IPython.display import Image, display, Video
from roboflow import Roboflow

In [9]:
model_9 = YOLO("runs/detect/train11/weights/best.pt")

In [19]:
!yolo detect val model=runs/detect/train11/weights/best.pt device=0 data=basketballDetection-27/data.yaml split=test

Ultralytics YOLOv8.0.228 🚀 Python-3.10.13 torch-2.1.1+cu121 CUDA:0 (Tesla V100-PCIE-32GB, 32501MiB)
Model summary (fused): 268 layers, 43609692 parameters, 0 gradients, 164.8 GFLOPs
val: Scanning /scratch/zw2688/Court_Vision_Model_Dev/basketballDetection-25/test
/scratch/zw2688/conda_env/yolonas/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all        336       1684      0.942      0.923      0.959      0.828
            basketball        336        475      0.896      0.825   

In [20]:
!yolo detect val model=runs/detect/train10/weights/best.pt device=0 data=basketballDetection-27/data.yaml split=test batch=128 workers=4

Ultralytics YOLOv8.0.228 🚀 Python-3.10.13 torch-2.1.1+cu121 CUDA:0 (Tesla V100-PCIE-32GB, 32501MiB)
Model summary (fused): 268 layers, 43609692 parameters, 0 gradients, 164.8 GFLOPs
val: Scanning /scratch/zw2688/Court_Vision_Model_Dev/basketballDetection-25/test
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all        336       1684      0.948      0.944      0.968      0.841
            basketball        336        475      0.921      0.879      0.936       0.76
                  hoop        336        378      0.936      0.975      0.974      0.819
                  made        336        211      0.996      0.943      0.973      0.864
                person        336        620      0.937      0.981      0.987      0.919
Speed: 1.8ms preprocess, 8.7ms inference, 0.0ms loss, 2.7ms postprocess per image
Results saved to runs/detect/val2
💡 Learn more at https://docs.ultralytics.com/modes/val


In [21]:
!yolo detect val model=runs/detect/train9/weights/best.pt device=0 data=basketballDetection-27/data.yaml split=test batch=128 workers=4

Ultralytics YOLOv8.0.228 🚀 Python-3.10.13 torch-2.1.1+cu121 CUDA:0 (Tesla V100-PCIE-32GB, 32501MiB)
Model summary (fused): 268 layers, 43609692 parameters, 0 gradients, 164.8 GFLOPs
val: Scanning /scratch/zw2688/Court_Vision_Model_Dev/basketballDetection-25/test
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all        336       1684      0.937      0.933      0.965      0.837
            basketball        336        475      0.878      0.857      0.922       0.74
                  hoop        336        378      0.945      0.959      0.974      0.818
                  made        336        211      0.995      0.938      0.974      0.869
                person        336        620      0.929      0.979      0.989      0.921
Speed: 2.0ms preprocess, 8.5ms inference, 0.0ms loss, 4.4ms postprocess per image
Results saved to runs/detect/val3
💡 Learn more at https://docs.ultralytics.com/modes/val


In [25]:
!yolo detect predict model=runs/detect/train10/weights/best.pt device=0 source=basketballDetection-27/test/images split=test batch=128 save=True

Ultralytics YOLOv8.0.228 🚀 Python-3.10.13 torch-2.1.1+cu121 CUDA:0 (Tesla V100-PCIE-32GB, 32501MiB)
Model summary (fused): 268 layers, 43609692 parameters, 0 gradients, 164.8 GFLOPs

image 1/886 /scratch/zw2688/Court_Vision_Model_Dev/basketballDetection-27/test/images/-154_jpg.rf.c9ed7923185079fa0ec9c53947eec5fb.jpg: 384x640 2 basketballs, 2 hoops, 6 persons, 87.4ms
image 2/886 /scratch/zw2688/Court_Vision_Model_Dev/basketballDetection-27/test/images/-81_jpg.rf.dae4272dcf6cf306b9ec745097cb63ab.jpg: 384x640 1 basketball, 1 hoop, 1 made, 1 person, 11.2ms
image 3/886 /scratch/zw2688/Court_Vision_Model_Dev/basketballDetection-27/test/images/-9_jpg.rf.3d59d27297a181fcb981f221f60a0812.jpg: 480x640 3 basketballs, 3 hoops, 1 made, 13 persons, 83.4ms
image 4/886 /scratch/zw2688/Court_Vision_Model_Dev/basketballDetection-27/test/images/-9_jpg.rf.5d0e65bfef22af5addff8b61fe266db7.jpg: 384x640 1 made, 1 person, 11.8ms
image 5/886 /scratch/zw2688/Court_Vision_Model_Dev/basketballDetection-27/test/im